In [14]:
import warnings
warnings.filterwarnings("ignore")

#!pip install pyyaml

In [15]:
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.layers import Input, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
#以下為tensorflow為2.0以下import方法
from keras.layers.normalization import BatchNormalization 
#以下為tensorflow為2.0以上import方法
#from tensorflow.keras.layers import BatchNormalization
from keras.models import Model
import keras.backend as K
from sklearn.metrics import log_loss
from scale_layer import Scale
import pandas as pd 
import numpy as np
import os
import keras
import tensorflow
import random
import cv2
import math
import seaborn as sns
from keras.preprocessing import image
#以下為tensorflow為2.0以下import方法
from keras.preprocessing.image import img_to_array
#以下為tensorflow為2.0以上import方法
#from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pickle

def densenet121_model(img_rows, img_cols, color_type=1, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.5, dropout_rate=0.0, weight_decay=1e-4, num_classes=None):
    '''
    DenseNet 121 Model for Keras

    Model Schema is based on 
    https://github.com/flyyufelix/DenseNet-Keras

    # Returns
        A Keras model instance.
    '''

    # Handle Dimension Ordering for different backends
    global concat_axis
    img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
    concat_axis = 3


    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6,12,24,16] # For DenseNet-121

    # Initial convolution
    x = Convolution2D(nb_filter, (7,7), strides=(2, 2), name='conv1', use_bias=False)(img_input)#109 109 64    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate)
        
        # Add transition_block
        x = transition_block(x, stage, nb_filter, dropout_rate=dropout_rate)
        nb_filter = int(nb_filter)
    print(x.shape)
    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate)
    
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis )(x)
    x = Activation('relu')(x)
    #print(type(x))
    x_fc = GlobalAveragePooling2D()(x)
    x_fc_feature = Dense(100)(x_fc)
    x_fc = Dense(4)(x_fc_feature)
    x_fc = Activation('softmax')(x_fc)
    model = Model(img_input, x_fc_feature)

    return model

In [16]:
def conv_block(x, stage, branch, nb_filter, dropout_rate=None):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout '''


    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(inter_channel, (1, 1), use_bias=False)(x)
    y=x
    #print(y.shape)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D((1, 1))(x)
    z=x
    #print(z.shape)
    x = Convolution2D(nb_filter, (3, 3), use_bias=False)(x)
    w=x
    #print(w.shape)
    #print(type(w))
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    return x

def transition_block(x, stage, nb_filter, dropout_rate=None):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout  '''


    x = BatchNormalization(axis=concat_axis)(x)
    x = Scale(axis=concat_axis)(x)
    x = Activation('relu')(x)
    x = Convolution2D(int(nb_filter), 1, 1, use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate)
        #concat_feat = Concatenate(axis=concat_axis)([concat_feat, x])
        my_concat = Lambda (lambda x: K.concatenate([x[0],x[1]],axis=concat_axis))
        concat_feat=my_concat([concat_feat, x])
        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [40]:
import os

import numpy as np
from keras.utils import np_utils
from keras import backend as K
from keras.datasets.cifar import load_batch
from keras.utils.data_utils import get_file



def load_data():
  data=[]
  labels=[]
  random.seed(42)
  imagePaths = sorted(list(os.listdir("dataset")))
  random.shuffle(imagePaths)
  #print(imagePaths)

  for img in imagePaths:
    path=sorted(list(os.listdir("dataset/"+img)))
    for i in path:
        image = cv2.imread("dataset/"+img+'/'+i)
        image = cv2.resize(image, (224,224))
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32)
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)
  data = np.array(data, dtype="float32") / 255.0
  labels = np.array(labels)
  mlb = LabelBinarizer()
  labels = mlb.fit_transform(labels)
  #print(labels[0])

  (xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.25,random_state=42)
  #print(xtrain.shape, xtest.shape)
  return xtrain, ytrain, xtest, ytest

load_data()

(array([[[[0.3372549 , 0.6039216 , 0.6313726 ],
          [0.3372549 , 0.6039216 , 0.6313726 ],
          [0.3372549 , 0.6039216 , 0.6313726 ],
          ...,
          [0.39215687, 0.6431373 , 0.6862745 ],
          [0.38431373, 0.63529414, 0.6784314 ],
          [0.38039216, 0.6313726 , 0.6745098 ]],
 
         [[0.34901962, 0.6156863 , 0.6431373 ],
          [0.34509805, 0.6117647 , 0.6392157 ],
          [0.34509805, 0.6117647 , 0.6392157 ],
          ...,
          [0.4       , 0.6509804 , 0.69411767],
          [0.39215687, 0.6431373 , 0.6862745 ],
          [0.38431373, 0.63529414, 0.6784314 ]],
 
         [[0.34901962, 0.6156863 , 0.6431373 ],
          [0.34901962, 0.6156863 , 0.6431373 ],
          [0.34901962, 0.6156863 , 0.6431373 ],
          ...,
          [0.40784314, 0.65882355, 0.7019608 ],
          [0.4       , 0.6509804 , 0.69411767],
          [0.39215687, 0.6431373 , 0.6862745 ]],
 
         ...,
 
         [[0.5019608 , 0.7607843 , 0.75686276],
          [0.49019

In [41]:
from keras.callbacks import ModelCheckpoint
if __name__ == '__main__':

    # Example to fine-tune on 3000 samples from Cifar10

    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes =4
    batch_size = 16 
    nb_epoch = 10

    # Load Cifar10 data. Please implement your own load_data() module for your own dataset
    X_train, Y_train, X_valid, Y_valid = load_data()

    # Load our model
    model = densenet121_model(img_rows=img_rows, img_cols=img_cols, color_type=channel, num_classes=num_classes)
    filepath="bestmodel.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # 全連接層倒數第二個節點名稱
    #layer_name = model.layers[-2].name  
    #intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
    fc_feature = model.predict(X_train)

(?, 6, 6, 1408)


In [32]:
print(fc_feature)

[[ 1.14127658e-02 -1.21184643e-02 -1.81281604e-02 ...  4.77510877e-03
  -2.57444615e-03  8.14055093e-05]
 [ 2.04815753e-02 -2.02146918e-02 -2.28317082e-02 ...  7.97661766e-03
  -4.83810063e-03 -3.87112144e-04]
 [ 1.37002207e-02 -1.38297845e-02 -1.87781565e-02 ...  3.29611916e-03
  -1.64013077e-03 -2.21179333e-03]
 ...
 [ 2.57849563e-02 -2.35291105e-02 -2.79193129e-02 ...  7.58999120e-03
  -4.48212307e-03 -3.17182485e-03]
 [ 1.37118846e-02 -1.34835187e-02 -2.01255977e-02 ...  5.52265719e-03
  -2.67987535e-03 -4.84235585e-04]
 [ 2.01405101e-02 -1.99767742e-02 -2.52299644e-02 ...  7.48742931e-03
  -4.58741374e-03 -6.43134117e-04]]


In [42]:
    #features = model.predict(X_train)
    #print(type(features))
    #print(features.shape)
    #將特徵儲存至csv
    features_shape = fc_feature.shape  # (420, 6, 6, 1920)
    #new_shape = (features_shape[0], np.prod(features_shape[1:]))  # (420, 6*6*1920)
    #reshaped_features = fc_feature.reshape(new_shape)
    print(reshaped_features)#顯示特徵
    print(reshaped_features.shape)#顯示特徵的維度
    #transposed_features = reshaped_features.transpose()
    pd.DataFrame(fc_feature).to_csv("Chickenpox.csv")

[[ 0.03084389  0.05717747 -0.01849313 ...  0.04381473 -0.01063802
  -0.01061617]
 [ 0.03182232  0.05674845 -0.01906522 ...  0.02787902 -0.0100878
  -0.01087387]
 [ 0.0327157   0.05984464 -0.01987855 ...  0.04182199 -0.01111143
  -0.01132184]
 ...
 [ 0.04015823  0.0653798  -0.02234492 ...  0.04488189 -0.01033035
  -0.01300179]
 [ 0.02994485  0.0583839  -0.0171214  ...  0.04230029 -0.0106154
  -0.00967765]
 [ 0.02592811  0.04959463 -0.01595737 ...  0.02826483 -0.00966826
  -0.00602989]]
(1697, 100)


In [27]:
print(fc_feature)

[[ 0.03084389  0.05717747 -0.01849313 ...  0.04381473 -0.01063802
  -0.01061617]
 [ 0.03182232  0.05674845 -0.01906522 ...  0.02787902 -0.0100878
  -0.01087387]
 [ 0.0327157   0.05984464 -0.01987855 ...  0.04182199 -0.01111143
  -0.01132184]
 ...
 [ 0.04015823  0.0653798  -0.02234492 ...  0.04488189 -0.01033035
  -0.01300179]
 [ 0.02994485  0.0583839  -0.0171214  ...  0.04230029 -0.0106154
  -0.00967765]
 [ 0.02592811  0.04959463 -0.01595737 ...  0.02826483 -0.00966826
  -0.00602989]]
